## Sudoku with Gurobi

Copyright 2020, Gurobi Optimization, LLC

The Sudoku board is a 9x9 grid, which is further divided into a 3x3 grid
of 3x3 grids.  Each cell in the grid must take a value from 0 to 9.
No two grid cells in the same row, column, or 3x3 subgrid may take the
same value.

In the MIP formulation, binary variables x[i,j,k] indicate whether
cell <i,j> takes value 'k'.  The constraints are as follows:
  1. Each cell must take exactly one value (sum_v x[i,j,k] = 1)
  2. Each value is used exactly once per row (sum_i x[i,j,k] = 1)
  3. Each value is used exactly once per column (sum_j x[i,j,k] = 1)
  4. Each value is used exactly once per 3x3 subgrid (sum_grid x[i,j,k] = 1)
  
***  

In [ ]:
# Import needed files
import math
import gurobipy as gp
from gurobipy import GRB

### Input the Sudoku puzzle

In [ ]:
# Read the "givens" via a text file. 
f = open('sudoku.txt')

# Read each row of the Sudoku puzzle as a string. 
grid = f.read().split()

# n is the number of cells per row of the puzzle. 
# s is the size of the inner squares. 
n = len(grid[0])
s = int(math.sqrt(n))

### Create our Gurobi BILP

In [ ]:
# Create our 3-D array of model variables
model = gp.Model('sudoku')

vars = model.addVars(n, n, n, vtype=GRB.BINARY, name='G')

### Create the constraints

In [ ]:
# Fix variables associated with cells whose values are pre-specified
for i in range(n):
    for j in range(n):
        if grid[i][j] != '.':
            k = int(grid[i][j]) - 1
            vars[i, j, k].LB = 1

# Each cell must take one value
model.addConstrs((vars.sum(i, j, '*') == 1
                 for i in range(n)
                 for j in range(n)), name='C')

# Each value appears once per row
model.addConstrs((vars.sum(i, '*', k) == 1
                 for i in range(n)
                 for k in range(n)), name='R')

# Each value appears once per column
model.addConstrs((vars.sum('*', j, k) == 1
                 for j in range(n)
                 for k in range(n)), name='C')

# Each value appears once per subgrid
model.addConstrs((
    gp.quicksum(vars[i, j, k] for i in range(p*s, (p+1)*s)
                for j in range(q*s, (q+1)*s)) == 1
    for k in range(n)
    for p in range(s)
    for q in range(s)), name='Sub')

# Set objective (this can be entirely omitted)
model.setObjective(0, GRB.MAXIMIZE)

# solve BILP with Gurobi
model.optimize()

# This is not necessary but does show you what the full BILP looks like
# You'll find it in the file sudoku.lp
model.write('sudoku.lp')


### Output the solution

In [ ]:
print('')
print('Solution:')
print('')

# Retrieve optimization result

solution = model.getAttr('X', vars)

for i in range(n):
    sol = ''
    for j in range(n):
        for k in range(n):
            if solution[i, j, k] > 0.5:
                sol += str(k+1)
    print(sol)